# Benchmark

> Fill in a module description here

In [ ]:
#| default_exp benchmark

In [ ]:
#| export
from typing import Optional, Callable, Dict, Any
import numpy as np
from identibench.core import BenchmarkSpec, TrainingContext, _load_raw_sequences_from_files
import time
import random

In [ ]:
#| hide
import shutil
from fastcore.test import test_eq, test_ne# Import nbdev testing functions

In [ ]:
#| exporti
def _run_default_test(
    spec: BenchmarkSpec, # The benchmark specification.
    model: Callable[[np.ndarray], np.ndarray] # The trained model (assumed callable for simulation).
    ) -> Dict[str, Any]:
    """
    Runs the default testing logic (simulation) using the spec's metric_func.
    
    Args:
        spec: The BenchmarkSpec.
        model: The trained model, expected to accept u_test and return y_pred.

    Returns:
        A dictionary containing the calculated metric score (e.g., {'metric_score': 0.123}).

    Raises:
        RuntimeError: If no test files are found.
        ValueError: If prediction shape mismatch occurs. 
        Exception: Propagates exceptions from model call or metric calculation.
    """
    test_dir = spec.dataset_path / 'test'
    # Use glob from the imported module
    test_files = sorted(list(test_dir.glob('*.hdf5'))) 
    
    if not test_files:
        # Let's raise an error here, as testing without test files is problematic
        raise RuntimeError(f"No test files found in {test_dir}") 

    all_scores = []
    for test_file in test_files:
        # Load the single sequence from the file
        # This now correctly calls the _load_raw_sequences_from_files defined above
        u_test, y_test, _ = next(_load_raw_sequences_from_files([test_file], spec.u_cols, spec.y_cols, spec.x_cols))
        
        # Run prediction
        y_pred = model(u_test)

        # Validate prediction shape
        if y_pred.shape != y_test.shape:
            # Use a standard ValueError
            raise ValueError(f"Prediction shape mismatch in {test_file.name}. Expected {y_test.shape}, got {y_pred.shape}")

        # Calculate metric (allow metric_func errors to propagate)
        score = spec.metric_func(y_test, y_pred)
        if not isinstance(score, (float, np.number)):
             print(f"Warning: Metric function for {spec.name} did not return a number (got {type(score)}). Attempting conversion.")
             try: score = float(score)
             except Exception as e: 
                 print(f"Warning: Could not convert metric score to float: {e}. Using NaN.")
                 score = np.nan
        all_scores.append(score)
            
    # Calculate average score
    if not all_scores:
        final_score = np.nan 
        print(f"Warning: No valid scores calculated for benchmark {spec.name}.")
    else:
        final_score = np.mean(all_scores).item() # Ensure scalar float

    return {'metric_score': final_score}

In [ ]:
#| export
def run_benchmark(
    spec: BenchmarkSpec, # The benchmark specification object.
    build_model: Callable[[TrainingContext], Callable], # User function to build and train the model.
    hyperparameters: Dict[str, Any], # Dictionary of hyperparameters for this run.
    seed: Optional[int] = None, # Optional random seed for reproducibility. If None, a random seed is generated.
    ) -> Dict[str, Any]:
    """
    Runs a single benchmark configuration.

    Handles dataset preparation, model training, testing (default simulation or custom),
    timing, and result aggregation. Allows exceptions to propagate.

    Args:
        spec: The BenchmarkSpec defining the benchmark.
        build_model: A function that takes a TrainingContext and returns a trained, callable model.
        hyperparameters: A dictionary containing hyperparameters for the build_model function.
        seed: An optional integer seed for reproducibility.

    Returns:
        A dictionary containing the results (metrics, timings, config).

    Raises:
        Exception: Propagates errors from dataset preparation, training, or testing phases.
    """
    start_time = time.monotonic()
    
    # 1. Initialization & Setup
    actual_seed = seed if seed is not None else random.randint(0, 2**32 - 1)
    
    results = {
        'benchmark_name': spec.name,
        'dataset_id': spec.dataset_id,
        'hyperparameters': hyperparameters,
        'seed': actual_seed,
        'training_time_seconds': np.nan,
        'test_time_seconds': np.nan,
    }

    # 2. Ensure Dataset (Allow exceptions to propagate)
    spec.ensure_dataset_exists() 

    # 3. Create Context
    # If core.py was separate, you'd import TrainingContext
    context = TrainingContext(spec=spec, hyperparameters=hyperparameters, seed=actual_seed) 

    # 4. Training Phase (Allow exceptions to propagate)
    train_start_time = time.monotonic()
    model = build_model(context) 
    train_end_time = time.monotonic()
    results['training_time_seconds'] = train_end_time - train_start_time

    if model is None:
        # Raise a standard error if model building fails silently
        raise RuntimeError(f"build_model for {spec.name} did not return a model.") 
        
    # 5. Testing & Evaluation Phase (Allow exceptions to propagate)
    test_start_time = time.monotonic()
    if spec.test_func is not None:
        # Use custom test function
        test_results = spec.test_func(spec, model)
        if not isinstance(test_results, dict):
             # Use standard error
             raise TypeError(f"Custom test_func for {spec.name} did not return a dictionary.") 
    else:
        # Use default test logic (simulation)
        test_results = _run_default_test(spec, model)
        
    test_end_time = time.monotonic()
    results['test_time_seconds'] = test_end_time - test_start_time
    
    # Merge test results into main results
    results.update(test_results) 
        
    # 6. Return successful results
    return results

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()